### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
#tf.reset_default_graph()
#tf.set_random_seed(42)

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/AIML/Lab/prices.csv')

### Check all columns in the dataset

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [6]:
data.shape

(851264, 7)

In [7]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date', 'symbol'], axis = 1, inplace = True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
dataSubset = data.iloc[:1000,:]

In [11]:
dataSubset.shape

(1000, 5)

In [12]:
dataSubset.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = dataSubset.loc[:,['open', 'low', 'high', 'volume']]
y = dataSubset.loc[:,'close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [15]:
print ("Shape: ", X_train.shape)
X_train.head()

Shape:  (800, 4)


,open,low,high,volume
675,58.180000,57.980000,58.820000,3897200.0
358,26.230000,26.120001,26.530001,1051100.0
159,122.550003,122.180000,124.129997,570700.0
533,43.889999,43.889999,45.500000,6010500.0
678,22.570000,22.530001,22.910000,12891700.0


In [16]:
print ("Shape: ", X_test.shape)
X_test.head()

Shape:  (200, 4)


,open,low,high,volume
249,122.449997,121.599998,123.070000,216600.0
353,59.130001,59.130001,60.070000,3280200.0
537,83.089996,82.669998,83.449997,3043700.0
424,45.459999,45.200001,46.020000,3553300.0
564,52.090000,52.090000,52.750000,1967200.0


In [17]:
print ("Shape: ", y_train.shape)
y_train.head()

Shape:  (800,)


675     58.180000
358     26.389999
159    122.949997
533     44.990002
678     22.879999
Name: close, dtype: float64

In [18]:
print ("Shape: ", y_test.shape)
y_test.head()

Shape:  (200,)


249    122.169998
353     59.410000
537     83.019997
424     46.009998
564     52.529999
Name: close, dtype: float64

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.layers.batch_normalization(x, training=True)

#Actual close
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [0]:
pred = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(pred-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [25]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8116.82
Training loss at step:  10  is  6647.621
Training loss at step:  20  is  5472.4526
Training loss at step:  30  is  3696.946
Training loss at step:  40  is  3696.8403
Training loss at step:  50  is  3696.7876
Training loss at step:  60  is  3696.7627
Training loss at step:  70  is  3696.734
Training loss at step:  80  is  3696.7249
Training loss at step:  90  is  3696.7131


In [26]:
print ('Final Train Loss: ', train_loss)

Final Train Loss:  3696.7092


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [27]:
sess.run(W)

array([[-1.3786758 ],
       [-1.4626927 ],
       [ 6.1736007 ],
       [ 0.16680612]], dtype=float32)

In [28]:
sess.run(b)

array([24.028618], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [29]:
#Calculate train_op and loss
_, test_loss = sess.run([train_op, loss],feed_dict={x:X_test, 
                                                        y_:y_test})
    
print ('Test Loss: ', test_loss)

Test Loss:  2226.1443


In [30]:
print ('Absolute mean square loss difference between training and testing loss: ', abs(train_loss - test_loss))

Absolute mean square loss difference between training and testing loss:  1470.565


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer=sgd_optimizer, loss='mse')

### Execute the model

In [32]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
800/800 [==============================] - 0s 199us/step - loss: 8170.7762
Epoch 2/100
800/800 [==============================] - 0s 32us/step - loss: 8057.7280
Epoch 3/100
800/800 [==============================] - 0s 27us/step - loss: 7951.6378
Epoch 4/100
800/800 [==============================] - 0s 28us/step - loss: 7833.5809
Epoch 5/100
800/800 [==============================] - 0s 29us/step - loss: 7683.8138
Epoch 6/100
800/800 [==============================] - 0s 29us/step - loss: 7513.9102
Epoch 7/100
800/800 [==============================] - 0s 29us/step - loss: 7321.0341
Epoch 8/100
800/800 [==============================] - 0s 31us/step - loss: 7105.6255
Epoch 9/100
800/800 [==============================] - 0s 29us/step - loss: 6850.1773
Epoch 10/100
800/800 [==============================] - 0s 32us/step - loss: 6592.4426
Epoch 11/100
800/800 [==============================] - 0s 30us/step - loss: 6312.4449
Epoch 12/100
800/800 [=============================

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
dataIris = pd.read_csv('/content/gdrive/My Drive/AIML/Lab/Iris.csv')

### Splitting the data into feature set and target set

In [34]:
print('Shape: ', dataIris.shape)
dataIris.head()

Shape:  (150, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [35]:
# As Id feature is not required, lets drop it.
dataIris.drop('Id', axis = 1, inplace = True)
dataIris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
X = dataIris.iloc[:, :-1]
y = dataIris.loc[:,'Species']

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(y.values)
y=le.transform(y)

In [0]:
y = pd.get_dummies(y)

In [39]:
y.head()

,0,1,2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [40]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Divide the dataset into Training and test (70:30)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [48]:
print(X_train.shape)
X_train.head()

(105, 4)


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
6,4.6,3.4,1.4,0.3
25,5.0,3.0,1.6,0.2
21,5.1,3.7,1.5,0.4
92,5.8,2.6,4.0,1.2
9,4.9,3.1,1.5,0.1


In [49]:
print(y_train.shape)
y_train.head()

(105, 3)


,0,1,2
6,1,0,0
25,1,0,0
21,1,0,0
92,0,1,0
9,1,0,0


### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model1 = tf.keras.models.Sequential()

#Normalize the data
model1.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))


#Add 1st hidden layer
model1.add(tf.keras.layers.Dense(10, activation='relu'))

#Add 2nd hidden layer
model1.add(tf.keras.layers.Dense(8, activation='relu'))


#Add Dense Layer which provides 10 Outputs after applying softmax
model1.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.01)

#Compile the model
model1.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [51]:
model1.fit(X_train,y_train,          
          validation_data=(X_test, y_test),
          epochs=50,
          batch_size=20)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 3ms/step - loss: 1.0233 - acc: 0.3714 - val_loss: 1.1796 - val_acc: 0.4000
Epoch 2/50
105/105 [==============================] - 0s 159us/step - loss: 1.0014 - acc: 0.3524 - val_loss: 1.1716 - val_acc: 0.4000
Epoch 3/50
105/105 [==============================] - 0s 138us/step - loss: 1.0004 - acc: 0.3524 - val_loss: 1.1708 - val_acc: 0.4000
Epoch 4/50
105/105 [==============================] - 0s 119us/step - loss: 0.9886 - acc: 0.4095 - val_loss: 1.1578 - val_acc: 0.4000
Epoch 5/50
105/105 [==============================] - 0s 103us/step - loss: 0.9762 - acc: 0.4190 - val_loss: 1.1673 - val_acc: 0.4000
Epoch 6/50
105/105 [==============================] - 0s 139us/step - loss: 0.9724 - acc: 0.4095 - val_loss: 1.1715 - val_acc: 0.4000
Epoch 7/50
105/105 [==============================] - 0s 110us/step - loss: 0.9718 - acc: 0.4286 - val_loss: 1.1456 - val_acc: 0.4000
Epoch 8/50
105/105 

### Report Accuracy of the predicted values

In [52]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 4)                 16        
_________________________________________________________________
dense_7 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________


In [88]:
acc_val = model1.evaluate(X_test, y_test)
print ('Loss: ', acc_val[0])
print ('Accuracy: ', acc_val[1])

45/45 [==============================] - 0s 187us/step
Loss:  0.5892430596881443
Accuracy:  0.8666666679912143


In [0]:
prediction = model1.predict(X_test)

In [69]:
prediction[0]

array([0.01653575, 0.22621399, 0.75725025], dtype=float32)

In [0]:
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y_test,1))

In [79]:
print('Correcness of predicted values:')
sess.run(correct_prediction)

Correcness of predicted values:


array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [77]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print ('Accuracy: ', sess.run(accuracy))

Accuracy:  0.8666667
